In [1]:
# Basic imports
import torch
import brambox as bb
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt

# Settings
bb.logger.setConsoleLevel('ERROR')             # Only show error log messages

In [2]:
# Load annotations
anno = bb.io.load('pandas', 'data/annotations.pkl')
det = bb.io.load('pandas', 'data/detections.pkl')
my_class = 'bicycle'

anno = anno[anno['class_label'] == my_class]
det = det[det['class_label'] == my_class]

print('annotations:')
display(anno.head())
print('detections:')
display(det.head())

annotations:


,image,class_label,id,x_top_left,y_top_left,width,height,occluded,truncated,lost,difficult,ignore
32,VOC2007/JPEGImages/000015,bicycle,0,77.0,136.0,284.0,223.0,0.0,0.0,False,False,False
52,VOC2007/JPEGImages/000038,bicycle,0,159.0,178.0,121.0,69.0,0.0,0.0,False,False,False
158,VOC2007/JPEGImages/000111,bicycle,0,333.0,192.0,30.0,19.0,0.0,0.0,False,True,True
159,VOC2007/JPEGImages/000111,bicycle,0,302.0,198.0,17.0,12.0,1.0,0.0,False,True,True
160,VOC2007/JPEGImages/000111,bicycle,0,242.0,197.0,19.0,14.0,1.0,0.0,False,True,True


detections:


,image,class_label,id,x_top_left,y_top_left,width,height,confidence
615,VOC2007/JPEGImages/000015,bicycle,0,72.610777,172.365812,57.235195,104.382781,0.001605
644,VOC2007/JPEGImages/000015,bicycle,0,80.927115,110.521243,278.009561,254.713113,0.971148
655,VOC2007/JPEGImages/000015,bicycle,0,127.984102,147.184665,168.010858,165.226349,0.007087
673,VOC2007/JPEGImages/000015,bicycle,0,66.094004,136.214000,127.340482,111.684066,0.002426
674,VOC2007/JPEGImages/000015,bicycle,0,57.884863,151.942161,91.332775,142.815232,0.004063


In [3]:
pr = bb.stat.pr(det, anno, 0.7)  # IoU threshold of 0.5
ap = bb.stat.ap(pr)

# Find detection threshold with maximal F1
f1 = bb.stat.fscore(pr)
threshold = bb.stat.peak(f1)

In [4]:
# Filter detections based on threshold (computed in previous step from F1-curve)
filtered_det = det[det.confidence >= threshold.confidence].copy()

# Compute TP,FP
tpfp_det = bb.stat.match_det(filtered_det, anno, 0.5)
# match_det() returns detections with a 'tp' and 'fp' column.
print('TP FP:')
display(tpfp_det.head())
display(tpfp_det[tpfp_det['fp']].sort_index())

# Compute FN
fn_anno = bb.stat.match_anno(filtered_det, anno, 0.5)
# match_anno() returns annotations with a 'detection' column that has the index of the matched detection
# We use this to filter unmatched annotations (NaN as index)
fn_anno = fn_anno[fn_anno.detection.isnull()]
print('FN:')
display(fn_anno.head())

TP FP:


,image,class_label,id,x_top_left,y_top_left,width,height,confidence,tp,fp
187637,VOC2007/JPEGImages/005000,bicycle,0,69.995743,165.145361,274.747793,228.859315,0.990578,True,False
232138,VOC2007/JPEGImages/006199,bicycle,0,174.508627,182.488588,236.720195,178.817621,0.982034,True,False
143044,VOC2007/JPEGImages/003851,bicycle,0,150.934403,325.497444,214.931561,173.768722,0.979906,True,False
353582,VOC2007/JPEGImages/009447,bicycle,0,42.953583,6.266933,349.035960,262.690306,0.977787,True,False
163932,VOC2007/JPEGImages/004395,bicycle,0,129.387012,149.986230,248.522172,152.275562,0.973709,True,False


,image,class_label,id,x_top_left,y_top_left,width,height,confidence,tp,fp
66864,VOC2007/JPEGImages/001781,bicycle,0,427.736869,55.376200,72.166264,223.371616,0.703741,False,True
125649,VOC2007/JPEGImages/003388,bicycle,0,196.482108,146.375674,261.556479,188.916170,0.652124,False,True
135826,VOC2007/JPEGImages/003650,bicycle,0,323.067078,199.635542,155.075220,180.827581,0.732909,False,True
173128,VOC2007/JPEGImages/004616,bicycle,0,31.561705,19.105636,438.720520,352.202489,0.617254,False,True
215233,VOC2007/JPEGImages/005745,bicycle,0,-14.240210,270.846092,323.588298,178.814705,0.700215,False,True
260260,VOC2007/JPEGImages/006993,bicycle,0,199.793009,69.375368,241.700741,252.270258,0.967542,False,True
260261,VOC2007/JPEGImages/006993,bicycle,0,40.504676,21.507795,270.923578,182.300568,0.589916,False,True
311963,VOC2007/JPEGImages/008362,bicycle,0,30.455112,233.880226,365.151332,96.492923,0.605812,False,True
319336,VOC2007/JPEGImages/008540,bicycle,0,305.806325,124.948868,115.829284,176.219298,0.661485,False,True
343151,VOC2007/JPEGImages/009152,bicycle,0,128.101917,229.837381,73.554300,89.407306,0.623029,False,True


FN:


,image,class_label,id,x_top_left,y_top_left,width,height,occluded,truncated,lost,difficult,ignore,detection,criteria
158,VOC2007/JPEGImages/000111,bicycle,0,333.0,192.0,30.0,19.0,0.0,0.0,False,True,True,NaN,0.0
159,VOC2007/JPEGImages/000111,bicycle,0,302.0,198.0,17.0,12.0,1.0,0.0,False,True,True,NaN,0.0
160,VOC2007/JPEGImages/000111,bicycle,0,242.0,197.0,19.0,14.0,1.0,0.0,False,True,True,NaN,0.0
161,VOC2007/JPEGImages/000111,bicycle,0,218.0,205.0,34.0,12.0,0.0,0.0,False,True,True,NaN,0.0
162,VOC2007/JPEGImages/000111,bicycle,0,174.0,192.0,17.0,18.0,1.0,0.0,False,True,True,NaN,0.0
